<a href="https://colab.research.google.com/github/dolDolSee/TF2.0/blob/main/alexnet_kerassequence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf

class Alexnet(tf.keras.Model):
    def __init__(self, kernel_regular=None, n_classes=10):
        super(Alexnet,self).__init__()
        
        self.conv_layer = tf.keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), padding="valid")
        self.activation_layer = tf.keras.layers.Activation("relu")
        self.batch_layer = tf.keras.layers.BatchNormalization()
        self.batch_layer1 = tf.keras.layers.BatchNormalization()
        self.batch_layer2 = tf.keras.layers.BatchNormalization()
        self.batch_layer3 = tf.keras.layers.BatchNormalization()
        self.batch_layer4 = tf.keras.layers.BatchNormalization()
        self.maxpooling_layer = tf.keras.layers.MaxPooling2D(pool_size=(3,3), strides=(2,2))
        
        self.conv2_layer = tf.keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding="same", kernel_regularizer=kernel_regular)
        
        self.conv3_layer = tf.keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="same", kernel_regularizer=kernel_regular)
        self.conv5_layer = tf.keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="same", kernel_regularizer=kernel_regular)
        
        self.conv4_layer = tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding="same", kernel_regularizer=kernel_regular)
        
        self.flatten_layer = tf.keras.layers.Flatten()
        self.Dense_layer = tf.keras.layers.Dense(4096, activation="relu")
        self.Dense_layer1 = tf.keras.layers.Dense(4096, activation="relu")
        self.drop_out_layer = tf.keras.layers.Dropout(0.5)
        self.output_layer = tf.keras.layers.Dense(n_classes, activation="softmax")
        
        
    def call(self, x):
        input = tf.reshape(x,[-1,128,128,3])
        x = self.conv_layer(input)
        x = self.activation_layer(x)
        x = self.batch_layer(x)
        x = self.maxpooling_layer(x)
        x = self.conv2_layer(x)
        x = self.activation_layer(x)
        x = self.batch_layer1(x)
        x = self.maxpooling_layer(x)
        x = self.conv3_layer(x)
        x = self.activation_layer(x)
        x = self.batch_layer2(x)
        x = self.conv5_layer(x)
        x = self.activation_layer(x)
        x = self.batch_layer3(x)
        x = self.conv4_layer(x)
        x = self.activation_layer(x)
        x = self.batch_layer4(x)
        x = self.maxpooling_layer(x)
        x = self.flatten_layer(x)
        x = self.Dense_layer(x)
        x = self.drop_out_layer(x)
        x = self.Dense_layer1(x)
        x = self.drop_out_layer(x)
        x = self.output_layer(x)
        return x

model = Alexnet(tf.keras.regularizers.l2(l2=1e-4), 10)
        
        

In [ ]:
from tensorflow.keras.datasets import cifar10

# 전체 6만개 데이터 중, 5만개는 학습 데이터용, 1만개는 테스트 데이터용으로 분리
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
print("train dataset shape:", train_images.shape, train_labels.shape)
print("test dataset shape:", test_images.shape, test_labels.shape)

In [ ]:
from tensorflow.keras.utils import to_categorical
train_labels.shape
to_categorical(train_labels)

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

import random as python_random
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import cifar10

def zero_one_scaler(image):
    return image/255.0

def get_preprocessed_ohe(images, labels, pre_func=None):
    # preprocessing 함수가 입력되면 이를 이용하여 image array를 scaling 적용.
    if pre_func is not None:
        images = pre_func(images)
    # OHE 적용    
    oh_labels = to_categorical(labels)
    return images, oh_labels


def get_train_valid_test_set(train_images, train_labels, test_images, test_labels, valid_size=0.15, random_state=2021):
    train_images, train_oh_labels = get_preprocessed_ohe(train_images, train_labels)
    test_images, test_oh_labels = get_preprocessed_ohe(test_images, test_labels)
    

    tr_images, val_images, tr_oh_labels, val_oh_labels = train_test_split(train_images, train_oh_labels, test_size=valid_size, random_state=random_state)
    
    return (tr_images, tr_oh_labels), (val_images, val_oh_labels), (test_images, test_oh_labels )

(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)
(tr_images, tr_oh_labels), (val_images, val_oh_labels), (test_images, test_oh_labels) = \
    get_train_valid_test_set(train_images, train_labels, test_images, test_labels, valid_size=0.2, random_state=2021)

print(tr_images.shape, tr_oh_labels.shape, val_images.shape, val_oh_labels.shape, test_images.shape, test_oh_labels.shape)

원본 이미지 크기를 증가시키는 Sequence Dataset생성

In [ ]:
IMAGE_SIZE = 128
BATCH_SIZE = 64

In [ ]:
from tensorflow.keras.utils import Sequence
import cv2
import sklearn

In [ ]:
class CIFAR_Dataset(Sequence):
    def __init__(self, images_array, labels, batch_size=BATCH_SIZE, augmentor=None, shuffle=False, pre_func=None):
        self.images_array = images_array
        self.labels = labels
        self.batch_size = batch_size
        self.augmentor = augmentor
        self.pre_func = pre_func
        self.shuffle = shuffle
        if self.shuffle:
            pass
    #batch size단위로 데이터를 몇번 가져와야하는지 입력해줌
    def __len__(self):
        return int(np.ceil(len(self.labels)/self.batch_size))
    # 인자로 몇번째 batch 인지를 나타내는 index를 입력하면 해당 순서에 해당하는 batch_size 만큼의 데이타를 가공하여 반환
    def __getitem__(self, index):
        images_fetch = self.images_array[index*self.batch_size:(index+1)*self.batch_size]
        if self.labels is not None:
            label_batch = self.labels[index*self.batch_size:(index+1)*self.batch_size]
        image_batch = np.zeros((images_fetch.shape[0],IMAGE_SIZE,IMAGE_SIZE,3), dtype="float32")
        for image_index in range(images_fetch.shape[0]):
            image = cv2.resize(images_fetch[image_index],(IMAGE_SIZE,IMAGE_SIZE))
            if self.augmentor is not None:
                image = self.augmentor(image=image)['image']
            if self.pre_func is not None:
                image = self.pre_func(image)
            image_batch[image_index] = image
        return image_batch, label_batch
    def on_epoch_end(self):
        if(self.shuffle):
            #print('epoch end')
            # 원본 image배열과 label를 쌍을 맞춰서 섞어준다. scikt learn의 utils.shuffle에서 해당 기능 제공
            self.images_array, self.labels = sklearn.utils.shuffle(self.images_array, self.labels)
        else:
            pass
        
        

In [ ]:
def zero_one_scaler(image):
    return image/255.0

tr_ds = CIFAR_Dataset(tr_images, tr_oh_labels, batch_size=BATCH_SIZE, augmentor=None, shuffle=True, pre_func=zero_one_scaler)
val_ds = CIFAR_Dataset(val_images, val_oh_labels, batch_size=BATCH_SIZE, augmentor=None, shuffle=False, pre_func=zero_one_scaler)

print(next(iter(tr_ds))[0].shape, next(iter(val_ds))[0].shape)
# print(next(iter(tr_ds))[1].shape, next(iter(val_ds))[1].shape)
# print(next(iter(tr_ds))[0][0])

In [ ]:
next(iter(tr_ds))[0].shape

In [ ]:
tf.reshape(next(iter(tr_ds))[0],[-1,128,128,3]).shape

In [ ]:
from tensorflow.keras.optimizers import Adam , RMSprop 
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
model.compile(optimizer=Adam(lr=0.001), loss="categorical_crossentropy", metrics=["accuracy"])

rlr_cb = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=5, mode="min", verbose=1)
ely_cb = EarlyStopping(monitor="val_loss", patience=10, mode="min", verbose=1)

history = model.fit(tr_ds, epochs=30,validation_data=val_ds, callbacks=[rlr_cb, ely_cb])

In [ ]:
test_ds = CIFAR_Dataset(test_images, test_oh_labels, batch_size=BATCH_SIZE, augmentor=None, shuffle=False, pre_func=zero_one_scaler)
model.evaluate(test_ds)